# Configuration

Setup the following:
- data directory
- env vars

In [5]:
%load_ext autoreload
%autoreload 2

In [9]:
import anthropic

In [25]:
import bizwiz
bizwiz.load_llm_env()

path_manager = bizwiz.PathManager(
    data_dir = "../data/"
)

chat_manager = bizwiz.ChatManager()


In [26]:
chat_manager.promt('testing if claude works')

"Hello! Yes, I'm functioning properly. Is there anything specific you'd like assistance with today?"